# Lab 6

In this lab, we'll be covering topic modeling - which moves us beyond the "bag of words" representation of text. Topic modeling (and related techniques)  can be thought of as a type of "dimensionality reduction" technique.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Part 1: Create a corpus

In [ ]:
import os
import sys
sys.path.append('..')

import lzma
import json
import pandas as pd
import numpy as np

from config import settings_base as settings
from config import utils

In [ ]:
# Get Case Data for California
compressed_file = utils.get_and_extract_from_bulk(jurisdiction="Delaware", 
                                                  data_format="json")

In [ ]:
# Assume we are dealing with json data (if data_format is changed to xml or
# change this cell's os.path.join line)
if not compressed_file.endswith('.xz'):
  compressed_file = os.path.join(compressed_file, "data", "data.jsonl.xz") 

In [ ]:
cases = []
print("File path:", compressed_file)
with lzma.open(compressed_file) as infile:
    for line in infile:
        record = json.loads(str(line, 'utf-8'))
        cases.append(record)

print("Case count: %s" % len(cases))

In [ ]:
df = pd.DataFrame(cases)
df.head()

In [ ]:
opinion_data = []
for case in cases:
    for opinion in case["casebody"]["data"]["opinions"]:
        temp = {}
        keys = list(case.keys())
        keys.remove('casebody')
        for key in keys:         
            temp[key] = case[key]
        keys = list(opinion.keys())
        for key in keys:         
            temp[key] = opinion[key]
        opinion_data.append(temp)
        
df = pd.DataFrame(opinion_data)
df["citations"] = df["citations"].apply(lambda x:x[0]['cite'])
df["court"] = df["court"].apply(lambda x:x['name'])
df["decision_date"] = df["decision_date"].apply(lambda x:int(x[:4]))
df = df[df['court'] =='Delaware Court of Chancery']
df = df.drop(["docket_number", "first_page", 
                                "last_page", "name",
                                "reporter", "volume", "jurisdiction"], axis=1)
df = df[["name_abbreviation", "decision_date", "court", "author", "type", "text"]]

In [ ]:
df.head()

In [ ]:
len(df)

# Part 2: Clustering algorithms

Clustering is an example of "unsupervised" machine learning - meaning that there is no human telling the computer the labels.

Classification - that we studied last time - was "supervised" machine learning - meaning that we knew the "labels" of data beforehand - and these labels were used to create our classification models. 

Clustering is useful when we don't know the classes in our data. See: [this image](https://miro.medium.com/max/1400/0*Xe3YnRNqb7AuIUdu.gif) for a basic idea of "clustering"



### **K-means Clustering**

K-Means is the granddaddy of clustering algorithms. Each data point (in this case, case) has to be a member of a distinct class. The "within-class" sum of squares has to be minimized - the  alogirthm tries to seperate clusters based on means of each "cluster." 

The problem with K-means is the classsical problem of "means" - that each cluster is sensetive to outliers.

See this image for a vizualization of the [iterative nature](https://media0.giphy.com/media/12vVAGkaqHUqCQ/giphy.gif?cid=790b76117b7e712fa0d37536e033c289c372b105a4d0447b&rid=giphy.gif&ct=g) of the K-means algorithm.

See this image for a representation of how it works with ["means"](https://upload.wikimedia.org/wikipedia/commons/e/ea/K-means_convergence.gif)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=0.01,
                             max_df=.9,  
                             max_features=1000,
                             stop_words='english',
                            ngram_range=(1,2))

In [ ]:
X = vectorizer.fit_transform(df.text)

In [ ]:
# Convert X to dense matrix, since the below clustering algorithms can only work with dense matrices
X_dense = X.todense()
X_dense

In [ ]:
# create 10 clusters of similar documents
from sklearn.cluster import KMeans

num_clusters = 10

km = KMeans(n_clusters=num_clusters)

km.fit(X)
doc_clusters = km.labels_.tolist()

In [ ]:
df['cluster'] = doc_clusters
df.head()

What does each cluster represent?

In [ ]:
df[df['cluster'] == 3]['text'][5]

**Silhouette Score**

Sillhoutte score is one of those techniuqes which allows us to choose the "optimal number" of clusters for the data.

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_score(X, km.labels_)

In [ ]:
sil_scores = []
for n in range(2, num_clusters):
    km = KMeans(n_clusters=n)
    km.fit(X)
    sil_scores.append(silhouette_score(X, km.labels_))

In [ ]:
import matplotlib.pyplot as plt 
plt.plot(range(2, num_clusters), sil_scores)
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.show()

In [ ]:
opt_sil_score = max(sil_scores[0:10])
sil_scores.index(opt_sil_score)
opt_num_cluster = range(2, num_clusters)[sil_scores.index(opt_sil_score)]
print('The optimal number of clusters is %s' %opt_num_cluster)

In [ ]:
km = KMeans(n_clusters=opt_num_cluster)
km.fit(X)
doc_clusters = km.labels_.tolist()

In [ ]:
df['cluster_mean'] = doc_clusters
df[df['cluster_mean']==1]['text']

### **K-Medoids**

K-medoid is similar to K-Means only that the difference is instead of means, we're working with centroids based on "medians" - which are less sensetive to outliers. 

Since medians have to be actual observations in the data, K-medoid allows us to pick an actual document which is, in a way most "representative" of a cluster. 

See [this picture](https://www.researchgate.net/publication/342871651/figure/fig1/AS:912165510864897@1594488613267/The-graphical-representation-of-the-difference-between-the-k-means-and-k-medoids.png) for an explanation of the difference.

In [ ]:
!pip install scikit-learn-extra

In [ ]:
from sklearn_extra.cluster import KMedoids

kmed = KMedoids(n_clusters=opt_num_cluster)
kmed.fit(X)
doc_clusters = kmed.labels_.tolist()

In [ ]:
df['cluster_med'] = doc_clusters
df.head()

In [ ]:
df[df['cluster_med']==1]['text']

In [ ]:
df[df['cluster_med']==1]['text'][10]

In [ ]:
df[df['cluster_med']==1]['text'][18]

### **DBSCAN**

DBSCAN  is a relatively new clustering algorithm (first published in 2014) 
 which stands for __"Density-Based Spatial Clustering of Applications with Noise"__. As the name suggests, DBSCAN uses "density" measure for points, rather than a simple means.

To see how DBSCAN works based on densities - see [this image](https://ml-explained.com/articles/dbscan-explained/dbscan.gif)

DBSCAN is very useful for non-circle/globular data - see [this image](https://miro.medium.com/max/1400/1*rfi9uHjGPdNgXgxe9xWvVw.png)

There are also different determinations of what means ["density"](https://dashee87.github.io/images/DBSCAN_search.gif) - as anything outside the "cluster" is deemed to be noise (or outlier) depending on the parametarization.


DBSCAN is a relatively new clustering algorithm (first published in 2014) 


In [ ]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.95, min_samples=5)
dbscan.fit(X)
db_clusters = dbscan.labels_

In [ ]:
df['cluster_db'] = db_clusters
df.head()

In [ ]:
df['cluster_db'].unique()

What does __-1__ cluster mean? 

Check  the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) - __"Cluster labels for each point in the dataset given to fit(). Noisy samples are given the label -1."__

In [ ]:
df[df['cluster_db']==1]['text']

### **Hierarchical DBSCAN**
Automatically chooses epsilon, performing DBSCAN over various epsilon values - and returns the result that gives the best stability over epsilon. For reference see [here](https://github.com/scikit-learn-contrib/hdbscan/).

In [ ]:
#!pip install hdbscan
#!pip install --upgrade numpy

In [ ]:
from hdbscan import HDBSCAN

hdbscan = HDBSCAN(min_cluster_size=5)
hdbscan.fit(X)
hdb_clusters = hdbscan.labels_

In [ ]:
df['cluster_hdb'] = hdb_clusters
df.head()

In [ ]:
df['cluster_hdb'].unique()

In [ ]:
df[df['cluster_hdb']==1]['text']

### **Hierarchical (Agglomerative) Clustering**
For Agglomerative clustering, each point starts as a cluster, and is combined based on distance to other points. At the end - you get "optimal" number of clusters (depending on where you define the cutoff).

See [this image](https://dashee87.github.io/images/hierarch.gif) for more detail

In [ ]:
from sklearn.cluster import AgglomerativeClustering

cluster = AgglomerativeClustering(n_clusters=opt_num_cluster, 
                                  affinity='euclidean', 
                                  linkage='ward')

cluster.fit_predict(X.toarray())

clusters = dbscan.labels_

In [ ]:
df['cluster_hie'] = clusters
df.head()

In [ ]:
df['cluster_hie'].unique()

In [ ]:
df[df['cluster_hie']==1]['text']

Summary - clustering is useful when we want to study how documents in our corpus are related. A set of documents could be a member of "Cluster 1" or "Cluster 2" - this can be useful.

However, because each document __has__ to be a part of the cluster (or noise) - this makes the determination of clusters not that intuitive. 

# Topic Modeling

Humans don't really think in terms of data points or "similarity measures." We mostly think of things in terms of parts-based approaches - your face is not a bunch of points, but is a combination of "the nose cluster", "eyes cluster," etc. 



## Part 3 - LDA

Latent Dirilect Allocation, or LDA, is an approach to model the distribution of words that appear in a body of text.

__"Latent"__ means hidden (nobody knows the exact topics in books)

__"Dirichlet"__ means [Dirichlet probability disturbiton](https://en.wikipedia.org/wiki/Dirichlet_distribution)  used for this algorithm (mathy stuff

__"Allocation"__ means allocating words to topics.

Basically, one can think of the LDA algorith mas just allocation of words to topics based on the Dirichlet distribution. 

* Unlike clustering, there is an assumption on the data - that "words" (data points) tend to appear together when certain topics are discussed. Thus, each topic "generates" a probability on the "words" that appear in it. 

* See [this image](https://miro.medium.com/max/800/1*pZo_IcxW1GVuH2vQKdoIMQ.jpeg) for an explanation of the intuition behind LDA

* See [this image](https://miro.medium.com/max/1200/1*jjhkii_JmvCEazFAzQIdGA.gif) for a representation of LDA output details.

* See [this video](https://youtu.be/MqPKguO5hDA) for a cool presentation of how LDA can be applied to other non-text domains.

If you think about it carefully, everything in our lives actually a "topic". 

LDA is different from clustering in that it's "generative" - meaning there's a Dirichlet probability that relates  certain words appearing in a certain "topic". 

For example, the word "Hogwarts" (0.9 probability), "magic" (0.8 probability), "wizardry" (0.7 probability) appear in a topic on "Harry Potter". 

In [ ]:
# vizualize the document term matrix from Part 2
X_matrix = pd.DataFrame(X_dense,
                       columns = vectorizer.get_feature_names())
X_matrix

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

# n_topics represents the number of topics you're training the LDA model to fit to
n_topics = 20

lda = LatentDirichletAllocation(n_components = n_topics, # how many topics we want 
                                max_iter = 10, # maximum learning iterations 
                                learning_method = 'online',
                                learning_offset = 80., 
                                total_samples = len(X_matrix),
                                random_state = 0)
lda.fit(X_matrix)

In [ ]:
#This is a function to print out the top words for each topic in a pretty way.
#Don't worry too much about understanding every line of this code.
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

print("\nTopics in LDA model:")
feature_names = vectorizer.get_feature_names()

# take a look at the print_top_words function above to get an understanding of what each paramter menas
print_top_words(lda, feature_names, 15) 

In [ ]:
# get the distribution array
topic_dist = lda.transform(X)
topic_dist



In [ ]:
topic_dist_df = pd.DataFrame(topic_dist)

In [ ]:
# Merge back in with the original dataframe.
pd.options.display.max_colwidth = 100

topic_dist_df = pd.DataFrame(topic_dist)

df = df.reset_index() ## need to reset index to merge properly)
df_w_topics = df.join(topic_dist_df)
df_w_topics.head()

__What is the most "representative topic" of topic 8?__

In [ ]:
topic_of_interest = 8

df_w_topics[['name_abbreviation', 
             'decision_date', 
             'text',
              topic_of_interest]].sort_values(by=[topic_of_interest], ascending=False)

In [ ]:
!pip install pyldavis

In [ ]:
# pyLDAvis is a package which allows you to view topic distribution of your text
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

lda_display = pyLDAvis.sklearn.prepare(lda, 
                                       X, 
                                       vectorizer)

pyLDAvis.save_html(lda_display, 'lda_visualization.html')
# See lda_visualization.html to explore the LDA based topics

lda_display

## Part 4 - NMF, PCA, SVD - matrix decomposition methods

### **Non-negative Matrix Factorization (NMF)**

Non-Negative Matrix Factorization (Lee and Seung, 1999) is a "Matrix decomposition" method with an imposed non-negativity constraint.     


Recall from the classification lab, that the Document-Term Matrix is full of zeroes - but it has __no negative values__ meaning we can use non-negative matrix factorization. 

The intuition is that we define our Document Term Matrix (X) as being composed of 2 other matrices (W) and (H).

Thus, [X ~= W * H](https://media.geeksforgeeks.org/wp-content/uploads/20210429213042/Intuition1-660x298.png)


See [this image](https://blog.acolyer.org/wp-content/uploads/2019/02/nmf-fig-1.jpeg?w=640) for how NMF was used initially for image processing.

See [this image](https://miro.medium.com/max/1400/1*Cdk8UXkHqkLxfPEFTNEU4A.jpeg) for details on the "matrix decomposition" part as it applies to text.






In [ ]:
X_matrix.head()

In [ ]:
vocab = np.array(vectorizer.get_feature_names())



# Define get_descriptor function which will show top words for a given topic
def get_descriptor( features, H, topic_index, top ):
    top_indices = np.argsort( H[topic_index,:] )[::-1]
    top_terms = []
    for term_index in top_indices[0:top]:
        top_terms.append( features[term_index] )
    return top_terms

# define get_top_documents function which will show us top cases associated with topics
def get_top_documents( cases, W, topic_index, top ):
    top_indices = np.argsort( W[:,topic_index] )[::-1]
    top_documents = []
    for doc_index in top_indices[0:top]:
        top_documents.append(cases[doc_index])
    return top_documents

In [ ]:
n_topics = 10
top_words = 10

# create NMF model
from sklearn import decomposition
model = decomposition.NMF(init = "nndsvd", 
                          n_components = n_topics)



In [ ]:
# apply the model and extract the two W and H matrices -> X ~= W*H 
W = model.fit_transform(X)
H = model.components_



In [ ]:
# show topics and words in those topics
descriptors = []
for topic_index in range( n_topics ):
    descriptors.append( get_descriptor( vocab, H, topic_index, top_words) )  # Top 10 words
    str_descriptor = ", ".join( descriptors[topic_index] )
    print("Topic %02d: %s" % ( topic_index+1, str_descriptor ) )

Most representative documents for a given topic

In [ ]:
case_names = df['name_abbreviation'].tolist()

topic_of_interest = 2
n_docs = 10

#Print top documents for a given topic
topic_documents = get_top_documents(case_names, W, topic_of_interest, n_docs) 
for i, doc in enumerate(topic_documents):
    print("%02d. %s" % ((i+1), doc))

### **SVD - singular value decomposition**

[SVD](https://www.sharetechnote.com/image/EngMath_Matrix_SVD_01_2.png) is another matrix decomposition technique - unlike NMF it gives 3 matrices instead of 2. 

In text analytic practice, it's not used as often as LDA or NMF for topic modeling. 

In [ ]:
def show_topics(a):
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
    topic_words = ([top_words(t) for t in a])
    return [' '.join(t) for t in topic_words]

In [ ]:
U, s, Vh = np.linalg.svd(X_matrix, full_matrices=False)
print(U.shape, s.shape, Vh.shape)

In [ ]:
num_top_words=8

def show_topics(a):
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
    topic_words = ([top_words(t) for t in a])
    return [' '.join(t) for t in topic_words]

show_topics(Vh[:10])

### Extra: Vizualizations - **PCA (Principle Component Analysis)** , __T-SNE__

Another classical matrix decomposition technique useful for summarizing high-dimensional data while preserving most information. PCA is more used in terms __visualizing__ the data


PCA can help us reduce the "dimensionality" of the data by finding the dimensions which explain most of the data - these "dimensions" are called "principal components (PC1 and PC2)." 

See [this image](https://miro.medium.com/max/800/1*ZXhPoYQIn-Y8mxoUpz5Ayw.gif) for an explanation of how it basically works

See this [this really good](https://stats.stackexchange.com/questions/2691/making-sense-of-principal-component-analysis-eigenvectors-eigenvalues) post explainig PCA

__T-SNE__ is another visualization method - but it is kinda problematic - see [this article](https://distill.pub/2016/misread-tsne/)

Nowadays, a newer algorithm called __UMAP__ is used more often for these types of visualizaitons - but agian, the problem of summarizing high-to-low dimensional representations is a [difficult one](https://miro.medium.com/max/1200/1*IHiZSKD019MrRmiyxaw2gg.gif). 

In [ ]:
# Principal Components
from sklearn.decomposition import PCA
pca = PCA(n_components=3,
          svd_solver='randomized')

Xpca = pca.fit_transform(X_matrix)
pca.explained_variance_ratio_

In [ ]:
# PCA Viz
plt.scatter(Xpca[:,0],Xpca[:,1], alpha=.1)
plt.show()

In [ ]:
#%% PCA 3D Viz
from mpl_toolkits.mplot3d import Axes3D
Axes3D(plt.figure()).scatter(Xpca[:,0],Xpca[:,1], Xpca[:,2], alpha=.1)
plt.show()

In [ ]:
# make components (dimensions) to explain 95% of variance
pca = PCA(n_components=.95)
X95 = pca.fit_transform(X_matrix)
pca.n_components_

In [ ]:
#%% MDS, Isomap, and T-SNE
from sklearn.manifold import MDS, Isomap, TSNE
mds = MDS(n_components=2)
Xmds = mds.fit_transform(X.toarray()[:500,:200])
Axes3D(plt.figure()).scatter(Xmds[:,0],Xmds[:,1], alpha=.3)

In [ ]:
#%% Isomap
iso = Isomap(n_components=2)
Xiso = iso.fit_transform(X[:500,:200])
Axes3D(plt.figure()).scatter(Xiso[:,0],Xiso[:,1], alpha=.3)

In [ ]:
#%% t-SNE
tsne = TSNE(n_components=2, n_iter=250)
Xtsne = tsne.fit_transform(X[:500,:200])
Axes3D(plt.figure()).scatter(Xtsne[:,0],Xtsne[:,1], alpha=.3)